In [43]:
import pandas as pd
import matplotlib.pyplot as plt
from numpy import array
from numpy import hstack
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Sequential
from keras.metrics import MeanAbsolutePercentageError

In [44]:
data = pd.read_csv("hwdata.csv")

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5473 entries, 0 to 5472
Data columns (total 15 columns):
Unnamed: 0    5473 non-null int64
datadate      5473 non-null int64
tic           5473 non-null object
conm          5473 non-null object
datacqtr      5472 non-null object
atq           4604 non-null float64
chq           1962 non-null float64
cogsq         4523 non-null float64
dlttq         4680 non-null float64
ibq           5432 non-null float64
invtq         4339 non-null float64
saleq         5430 non-null float64
xrdq          1132 non-null float64
xsgaq         3068 non-null float64
prccq         5473 non-null float64
dtypes: float64(10), int64(2), object(3)
memory usage: 641.5+ KB


In [46]:
data['stock_return'] = (data.groupby('tic')['prccq']
                                  .apply(pd.Series.pct_change))

In [47]:
data.head()

,Unnamed: 0,datadate,tic,conm,datacqtr,atq,chq,cogsq,dlttq,ibq,invtq,saleq,xrdq,xsgaq,prccq,stock_return
0,0,19620331,BA,BOEING CO,1962Q1,NaN,NaN,NaN,NaN,8.041,NaN,466.8,NaN,NaN,50.1250,NaN
1,1,19620630,BA,BOEING CO,1962Q2,NaN,NaN,NaN,NaN,7.439,NaN,366.9,NaN,NaN,36.7500,-0.266833
2,2,19620930,BA,BOEING CO,1962Q3,NaN,NaN,NaN,NaN,6.099,NaN,434.7,NaN,NaN,37.9999,0.034011
3,3,19621231,BA,BOEING CO,1962Q4,NaN,NaN,NaN,NaN,5.575,NaN,500.1,NaN,NaN,37.1249,-0.023026
4,4,19630331,BA,BOEING CO,1963Q1,NaN,NaN,NaN,NaN,4.410,NaN,483.8,NaN,NaN,36.2499,-0.023569


In [48]:
data['datadate'] = pd.to_datetime(data['datadate'], format='%Y%m%d') 

In [49]:
data["quarter"] = data['datadate'].dt.quarter

In [50]:
data['ibq'] = data.groupby('tic')['ibq'].transform(lambda v: v.bfill())
data['saleq'] = data.groupby('tic')['saleq'].transform(lambda v: v.bfill())

In [51]:
data['stock_return'] = (data.groupby('tic')['prccq']
                                  .apply(pd.Series.pct_change))

In [52]:
data1 = data.sort_values(by = ['tic', 'datacqtr']).reset_index(drop = True)

In [53]:
data1 = data1[data1['stock_return'].notnull()] #Remove the first (null) of each company

In [54]:
tics = list(data1['tic'].unique())

In [55]:
evalresults = pd.DataFrame()
#scaler = MinMaxScaler()
for ticsymbol in tics:
    individual = data1[data1['tic'] == ticsymbol]
    # split a multivariate sequence into samples
    def split_sequences(sequences, n_steps):
        X, y = list(), list()
        for i in range(len(sequences)):
            # find the end of this pattern
            end_ix = i + n_steps
            # check if we are beyond the dataset
            if end_ix > len(sequences)-1:
                break
            # gather input and output parts of the pattern
            seq_x= sequences[i:end_ix, :]
            X.append(seq_x)

        return array(X)

    # define input sequence

    #data2 =data1[['Quarter','prccq','saleq','ibq']]
    in_seq1 = array(data1.quarter.tolist())
    in_seq2 = array(data1.stock_return.tolist())
    in_seq3 = array(data1.ibq.tolist())
    in_seq4 = array(data1.saleq.tolist())

    out_seq = array(data1.stock_return.tolist())

    # convert to [rows, columns] structure
    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    in_seq3 = in_seq2.reshape((len(in_seq3), 1))
    in_seq4 = in_seq2.reshape((len(in_seq4), 1))

    out_seq = out_seq.reshape((len(out_seq), 1))

    # horizontally stack columns
    dataset = hstack((in_seq1, in_seq2, in_seq3, in_seq4))
    # choose a number of time steps
    n_steps = 8

    # convert into input/output
    X = split_sequences(dataset, n_steps)
    y = out_seq[n_steps:]

    test_size = int(0.2 * len(X))           
    X_train, X_test, y_train, y_test = X[:-test_size], X[-test_size:], y[:-test_size], y[-test_size:]
    # the dataset knows the number of features, e.g. 2
    n_features = X.shape[2]

    # define model
    model = Sequential()
    model.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(n_steps, n_features)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=[MeanAbsolutePercentageError()])
    # fit model
    history = model.fit(X_train, y_train, epochs=100, batch_size=10,validation_data=(X_test, y_test))
    
    evalresults[ticsymbol + ' MAPE'] = history.history['mean_absolute_percentage_error']
    evalresults[ticsymbol + ' VAL_MAPE'] = history.history['val_mean_absolute_percentage_error']

Train on 4352 samples, validate on 1088 samples
Epoch 1/100
4352/4352 [==============================] - 1s 185us/step - loss: 0.0270 - mean_absolute_percentage_error: 589517.2500 - val_loss: 0.0398 - val_mean_absolute_percentage_error: 41480.1758
Epoch 2/100
4352/4352 [==============================] - 1s 128us/step - loss: 0.0253 - mean_absolute_percentage_error: 452594.4375 - val_loss: 0.0387 - val_mean_absolute_percentage_error: 105600.2109
Epoch 3/100
4352/4352 [==============================] - 1s 122us/step - loss: 0.0254 - mean_absolute_percentage_error: 418526.0000 - val_loss: 0.0410 - val_mean_absolute_percentage_error: 129439.9844
Epoch 4/100
4352/4352 [==============================] - 1s 127us/step - loss: 0.0250 - mean_absolute_percentage_error: 450345.4375 - val_loss: 0.0379 - val_mean_absolute_percentage_error: 34394.2773
Epoch 5/100
4352/4352 [==============================] - 1s 122us/step - loss: 0.0243 - mean_absolute_percentage_error: 303302.3125 - val_loss: 0.0379

4352/4352 [==============================] - 1s 117us/step - loss: 0.0195 - mean_absolute_percentage_error: 292672.0000 - val_loss: 0.0437 - val_mean_absolute_percentage_error: 125910.7812
Epoch 42/100
4352/4352 [==============================] - 1s 119us/step - loss: 0.0195 - mean_absolute_percentage_error: 299924.1250 - val_loss: 0.0454 - val_mean_absolute_percentage_error: 107586.1875
Epoch 43/100
4352/4352 [==============================] - 1s 122us/step - loss: 0.0193 - mean_absolute_percentage_error: 288136.2500 - val_loss: 0.0457 - val_mean_absolute_percentage_error: 92510.6172
Epoch 44/100
4352/4352 [==============================] - 1s 125us/step - loss: 0.0191 - mean_absolute_percentage_error: 285705.5625 - val_loss: 0.0457 - val_mean_absolute_percentage_error: 137987.0938
Epoch 45/100
4352/4352 [==============================] - 1s 119us/step - loss: 0.0190 - mean_absolute_percentage_error: 269862.7812 - val_loss: 0.0427 - val_mean_absolute_percentage_error: 105396.0156
Epoc

Epoch 82/100
4352/4352 [==============================] - 1s 128us/step - loss: 0.0160 - mean_absolute_percentage_error: 376514.0000 - val_loss: 0.0545 - val_mean_absolute_percentage_error: 191219.8125
Epoch 83/100
4352/4352 [==============================] - 1s 169us/step - loss: 0.0161 - mean_absolute_percentage_error: 403032.6562 - val_loss: 0.0520 - val_mean_absolute_percentage_error: 144299.1250
Epoch 84/100
4352/4352 [==============================] - 1s 138us/step - loss: 0.0160 - mean_absolute_percentage_error: 341642.3125 - val_loss: 0.0541 - val_mean_absolute_percentage_error: 250608.9375
Epoch 85/100
4352/4352 [==============================] - 1s 129us/step - loss: 0.0160 - mean_absolute_percentage_error: 447023.2500 - val_loss: 0.0525 - val_mean_absolute_percentage_error: 205021.7812
Epoch 86/100
4352/4352 [==============================] - 0s 107us/step - loss: 0.0161 - mean_absolute_percentage_error: 407603.3750 - val_loss: 0.0540 - val_mean_absolute_percentage_error: 21

4352/4352 [==============================] - 1s 154us/step - loss: 0.0218 - mean_absolute_percentage_error: 227084.9375 - val_loss: 0.0395 - val_mean_absolute_percentage_error: 84231.1094
Epoch 23/100
4352/4352 [==============================] - 1s 141us/step - loss: 0.0215 - mean_absolute_percentage_error: 187675.7188 - val_loss: 0.0410 - val_mean_absolute_percentage_error: 149665.1094
Epoch 24/100
4352/4352 [==============================] - 1s 141us/step - loss: 0.0216 - mean_absolute_percentage_error: 250409.7656 - val_loss: 0.0395 - val_mean_absolute_percentage_error: 69398.6094
Epoch 25/100
4352/4352 [==============================] - 1s 144us/step - loss: 0.0214 - mean_absolute_percentage_error: 274426.6250 - val_loss: 0.0397 - val_mean_absolute_percentage_error: 41980.3203
Epoch 26/100
4352/4352 [==============================] - 1s 146us/step - loss: 0.0213 - mean_absolute_percentage_error: 233203.2812 - val_loss: 0.0396 - val_mean_absolute_percentage_error: 23404.4766
Epoch 2

4352/4352 [==============================] - 1s 144us/step - loss: 0.0180 - mean_absolute_percentage_error: 334510.8125 - val_loss: 0.0436 - val_mean_absolute_percentage_error: 100318.5938
Epoch 63/100
4352/4352 [==============================] - 1s 148us/step - loss: 0.0178 - mean_absolute_percentage_error: 311979.5312 - val_loss: 0.0451 - val_mean_absolute_percentage_error: 83065.9297
Epoch 64/100
4352/4352 [==============================] - 1s 143us/step - loss: 0.0177 - mean_absolute_percentage_error: 376770.0312 - val_loss: 0.0449 - val_mean_absolute_percentage_error: 63888.1406
Epoch 65/100
4352/4352 [==============================] - 1s 139us/step - loss: 0.0178 - mean_absolute_percentage_error: 328102.4375 - val_loss: 0.0470 - val_mean_absolute_percentage_error: 66260.8203
Epoch 66/100
4352/4352 [==============================] - 1s 139us/step - loss: 0.0176 - mean_absolute_percentage_error: 357151.6562 - val_loss: 0.0450 - val_mean_absolute_percentage_error: 114547.0000
Epoch 

...

4352/4352 [==============================] - 1s 135us/step - loss: 0.0211 - mean_absolute_percentage_error: 236031.4844 - val_loss: 0.0380 - val_mean_absolute_percentage_error: 68642.2266
Epoch 37/100
4352/4352 [==============================] - 1s 131us/step - loss: 0.0210 - mean_absolute_percentage_error: 215777.7188 - val_loss: 0.0385 - val_mean_absolute_percentage_error: 59711.2617
Epoch 38/100
4352/4352 [==============================] - 1s 129us/step - loss: 0.0208 - mean_absolute_percentage_error: 277149.5000 - val_loss: 0.0388 - val_mean_absolute_percentage_error: 44561.5430
Epoch 39/100
4352/4352 [==============================] - 1s 132us/step - loss: 0.0207 - mean_absolute_percentage_error: 252806.4688 - val_loss: 0.0379 - val_mean_absolute_percentage_error: 62564.2266
Epoch 40/100
4352/4352 [==============================] - 1s 126us/step - loss: 0.0205 - mean_absolute_percentage_error: 275190.0625 - val_loss: 0.0394 - val_mean_absolute_percentage_error: 41284.5742
Epoch 41

Epoch 77/100
4352/4352 [==============================] - 1s 154us/step - loss: 0.0171 - mean_absolute_percentage_error: 291304.6875 - val_loss: 0.0478 - val_mean_absolute_percentage_error: 132373.5156
Epoch 78/100
4352/4352 [==============================] - 1s 131us/step - loss: 0.0170 - mean_absolute_percentage_error: 355158.2812 - val_loss: 0.0460 - val_mean_absolute_percentage_error: 159347.7812
Epoch 79/100
4352/4352 [==============================] - 1s 136us/step - loss: 0.0173 - mean_absolute_percentage_error: 360016.7188 - val_loss: 0.0492 - val_mean_absolute_percentage_error: 139924.9062
Epoch 80/100
4352/4352 [==============================] - 1s 139us/step - loss: 0.0170 - mean_absolute_percentage_error: 342752.6562 - val_loss: 0.0489 - val_mean_absolute_percentage_error: 102291.5625
Epoch 81/100
4352/4352 [==============================] - 1s 136us/step - loss: 0.0168 - mean_absolute_percentage_error: 332398.9688 - val_loss: 0.0470 - val_mean_absolute_percentage_error: 15

4352/4352 [==============================] - 1s 118us/step - loss: 0.0225 - mean_absolute_percentage_error: 210281.0469 - val_loss: 0.0380 - val_mean_absolute_percentage_error: 63181.5742
Epoch 18/100
4352/4352 [==============================] - 1s 120us/step - loss: 0.0226 - mean_absolute_percentage_error: 224526.4062 - val_loss: 0.0381 - val_mean_absolute_percentage_error: 88715.1797
Epoch 19/100
4352/4352 [==============================] - 1s 122us/step - loss: 0.0222 - mean_absolute_percentage_error: 191791.5781 - val_loss: 0.0377 - val_mean_absolute_percentage_error: 66976.1641
Epoch 20/100
4352/4352 [==============================] - 1s 123us/step - loss: 0.0221 - mean_absolute_percentage_error: 252529.0312 - val_loss: 0.0393 - val_mean_absolute_percentage_error: 83960.3047
Epoch 21/100
4352/4352 [==============================] - 1s 121us/step - loss: 0.0220 - mean_absolute_percentage_error: 186885.7031 - val_loss: 0.0381 - val_mean_absolute_percentage_error: 69201.2656
Epoch 22

Epoch 58/100
4352/4352 [==============================] - 0s 113us/step - loss: 0.0174 - mean_absolute_percentage_error: 383745.1875 - val_loss: 0.0431 - val_mean_absolute_percentage_error: 123809.5703
Epoch 59/100
4352/4352 [==============================] - 1s 135us/step - loss: 0.0174 - mean_absolute_percentage_error: 414078.8438 - val_loss: 0.0425 - val_mean_absolute_percentage_error: 186806.3281
Epoch 60/100
4352/4352 [==============================] - 1s 119us/step - loss: 0.0173 - mean_absolute_percentage_error: 374486.6875 - val_loss: 0.0440 - val_mean_absolute_percentage_error: 182820.4219
Epoch 61/100
4352/4352 [==============================] - 0s 114us/step - loss: 0.0173 - mean_absolute_percentage_error: 353454.2500 - val_loss: 0.0462 - val_mean_absolute_percentage_error: 207067.6250
Epoch 62/100
4352/4352 [==============================] - 1s 125us/step - loss: 0.0173 - mean_absolute_percentage_error: 395767.1250 - val_loss: 0.0439 - val_mean_absolute_percentage_error: 18

4352/4352 [==============================] - 1s 148us/step - loss: 0.0156 - mean_absolute_percentage_error: 472814.3438 - val_loss: 0.0512 - val_mean_absolute_percentage_error: 93937.8906
Epoch 99/100
4352/4352 [==============================] - 1s 256us/step - loss: 0.0153 - mean_absolute_percentage_error: 441863.3125 - val_loss: 0.0523 - val_mean_absolute_percentage_error: 141917.8594
Epoch 100/100
4352/4352 [==============================] - 1s 213us/step - loss: 0.0153 - mean_absolute_percentage_error: 452831.9375 - val_loss: 0.0519 - val_mean_absolute_percentage_error: 128643.3516


In [56]:
returnresults = evalresults
returndescription = returnresults.describe()
returnresults.to_csv('CNN Stock Return Results.csv')
returndescription.to_csv('CNN Stock Return Description.csv')

In [34]:
returnresults

,BA MAPE,BA VAL_MAPE,CMA MAPE,CMA VAL_MAPE,COP MAPE,COP VAL_MAPE,DIS MAPE,DIS VAL_MAPE,DTE MAPE,DTE VAL_MAPE,...,SLB MAPE,SLB VAL_MAPE,TMO MAPE,TMO VAL_MAPE,VFC MAPE,VFC VAL_MAPE,XOM MAPE,XOM VAL_MAPE,XRX MAPE,XRX VAL_MAPE
0,463715.156250,156800.500000,519686.50000,80695.859375,607575.37500,70020.828125,760132.87500,100255.812500,620819.06250,183742.500000,...,582989.06250,126411.132812,516156.281250,178090.406250,539794.000000,67950.851562,425837.28125,293774.468750,409558.062500,82986.671875
1,356114.468750,29549.162109,308601.40625,80180.335938,367800.46875,138093.921875,338882.43750,73584.289062,509053.46875,185699.937500,...,390072.53125,53579.656250,446662.125000,187941.265625,375128.406250,63842.781250,355747.90625,188909.093750,249511.906250,48185.164062
2,299862.687500,52816.218750,337437.15625,70649.109375,381217.65625,100100.921875,295233.03125,97304.734375,358913.84375,120451.312500,...,298071.46875,110375.648438,445111.343750,117279.718750,317228.531250,40297.406250,356342.93750,51449.421875,256402.890625,80788.273438
3,242847.578125,153054.437500,299379.37500,13738.843750,291645.71875,83442.453125,368639.18750,44047.179688,320872.68750,87556.312500,...,309597.18750,139646.750000,386036.468750,46663.214844,253837.265625,258944.140625,274067.96875,111802.273438,256816.609375,74300.281250
4,273062.343750,66024.656250,276771.62500,81080.929688,288480.25000,82840.046875,368522.03125,260307.375000,333287.62500,83997.687500,...,341233.25000,217314.703125,303736.375000,31967.796875,388397.468750,48842.410156,253893.21875,49642.851562,263013.562500,91050.187500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,249827.437500,133900.625000,373621.93750,87351.109375,376957.71875,134580.140625,275638.68750,142345.312500,308874.21875,90515.750000,...,371670.90625,100430.921875,261520.843750,136265.781250,371463.468750,104761.492188,448667.18750,151847.500000,318179.656250,233119.234375
96,315440.250000,219598.046875,347555.37500,71467.664062,357342.43750,85348.453125,356024.43750,170904.218750,350644.18750,77936.132812,...,322788.25000,91440.031250,258910.984375,131089.250000,338704.593750,144748.078125,410349.50000,105688.867188,360095.437500,113342.132812
97,229545.203125,144913.468750,265707.93750,72621.460938,321058.68750,81204.632812,305616.09375,152794.328125,299066.53125,109366.187500,...,343464.28125,95718.437500,309354.562500,162648.515625,341662.156250,86469.000000,433405.59375,109577.445312,354101.656250,177954.703125
98,304882.250000,124217.468750,333626.93750,36521.902344,413456.90625,54574.914062,307039.25000,167887.703125,308520.03125,65962.406250,...,301839.56250,76384.812500,319559.093750,156972.000000,333614.812500,96199.828125,441482.87500,229561.234375,346307.093750,124885.632812


In [35]:
returndescription

,BA MAPE,BA VAL_MAPE,CMA MAPE,CMA VAL_MAPE,COP MAPE,COP VAL_MAPE,DIS MAPE,DIS VAL_MAPE,DTE MAPE,DTE VAL_MAPE,...,SLB MAPE,SLB VAL_MAPE,TMO MAPE,TMO VAL_MAPE,VFC MAPE,VFC VAL_MAPE,XOM MAPE,XOM VAL_MAPE,XRX MAPE,XRX VAL_MAPE
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,259418.034688,106106.678496,290397.735938,116331.897383,280443.644688,99901.658730,272202.190312,120536.040605,272192.363594,95434.739668,...,272805.929063,99329.296914,240219.769219,99702.280767,278675.700000,120360.440566,302655.165156,113331.272930,280196.550312,91980.127588
std,51627.181643,40573.879264,55608.734471,49146.431489,67934.129076,45576.490622,73399.415318,45806.473706,60748.523824,34816.961302,...,59478.335235,34789.287239,61726.964107,42920.873734,61107.557197,40756.010227,76055.891336,49716.369962,45035.973655,42733.326996
min,155336.546875,25021.308594,183124.484375,10076.820312,165246.781250,29315.787109,168659.890625,30311.630859,182269.765625,27033.138672,...,182876.546875,41487.992188,166630.468750,8120.207520,185265.250000,28584.509766,176845.859375,39483.917969,197382.687500,15720.760742
25%,211643.906250,70819.091797,253159.910156,80566.978516,232790.089844,62855.949219,222348.371094,77463.742188,238862.433594,70880.429688,...,229402.179688,75150.806641,193118.097656,68164.414062,228174.816406,93924.142578,237242.820312,78857.455078,250114.921875,60639.479492
50%,263370.921875,112258.277344,298192.015625,109043.535156,277612.203125,94189.878906,272515.687500,127284.324219,266108.109375,92898.750000,...,267177.796875,93756.644531,229234.578125,100203.136719,274181.046875,123732.714844,311422.718750,104359.046875,268188.203125,80520.949219
75%,297229.484375,133803.933594,324831.289062,149747.183594,322355.492188,129111.589844,307931.015625,157874.464844,301266.453125,115741.177734,...,312097.515625,119556.751953,271330.039062,125724.240234,320821.890625,149847.890625,362419.648438,135783.914062,311382.648438,119021.421875
max,463715.156250,219598.046875,519686.500000,237236.515625,607575.375000,258755.937500,760132.875000,260307.375000,620819.062500,218779.546875,...,582989.062500,217314.703125,516156.281250,217299.281250,539794.000000,258944.140625,450218.250000,293774.468750,409558.062500,233119.234375
